In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings, copy
from collections import namedtuple, deque
from typing import NamedTuple, List, Any
import operator
import array
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 150, 'font.size': 12, 'axes.titlesize': 'large'})

# --- Utility Functions ---
def note(msg, **kwargs):
    """Prints a formatted message with a notebook icon."""
    formatted_msg = textwrap.fill(msg, width=100, subsequent_indent='   ')
    print(f"\n📝 {formatted_msg}", **kwargs)

def sec(title):
    """Prints a formatted section title for code blocks."""
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 1: Foundations
## Chapter 1.5: Sequence Data Structures: Lists and Tuples

### Introduction: Sequences, Mutability, and Semantic Meaning

Economic analysis is fundamentally concerned with ordered data: time series of asset prices, panel data observations for a set of individuals, or a sequence of policy actions. Python's primary built-in tools for managing such ordered data are its **sequence types**, with the `list` and `tuple` being the most fundamental.

The crucial technical distinction between them is **mutability**: a `list` is mutable (its contents and size can be changed after creation), while a `tuple` is immutable. However, this technical difference reflects a deeper semantic distinction that should guide their proper use. The choice between a `list` and a `tuple` is a design decision that communicates intent to the reader of your code.

- **Lists: A Collection of Peers.** A list is best understood as a container for a **homogeneous sequence**, where the items are of the same conceptual type and the number of items is arbitrary and may vary. The defining characteristic is that each element is a peer of the others. Examples include a series of stock returns `[0.01, -0.02, 0.005]`, a collection of agent objects in a simulation, or a list of country names `['USA', 'DEU', 'JPN']`. The list's mutability is a key feature, allowing you to add, remove, or reorder these peers as needed.

- **Tuples: A Composite Record.** A tuple is best understood as a single, **heterogeneous record**, where each position has a specific, implicit meaning and the structure is fixed. The items may be of different types, and their order is a crucial part of the record's definition. Think of a tuple as a single row from a spreadsheet or a single observation from a dataset: `(agent_id, wealth, age)` or `(country_code, year, gdp_growth)`. Its immutability is a key feature, ensuring the integrity and atomicity of the record. You wouldn't want to accidentally delete the `year` from that observation.

This semantic distinction has practical consequences for API design. Consider a function to calculate portfolio returns:
```python
def calculate_portfolio_return(assets: List[Tuple[str, float, float]]) -> float:
    # ... implementation ...
```
The type hint `List[Tuple[str, float, float]]` immediately communicates a wealth of information. It tells the user the function expects a **list** of assets, meaning it can handle any number of them. It also specifies that each individual asset should be represented as a **tuple**—a fixed record containing a ticker (string), a weight (float), and a return (float). This is far more expressive and less error-prone than simply hinting `List[Any]`. Choosing the right data structure is not just a technicality; it is a way of conveying intent. Understanding this distinction, and the performance characteristics of the underlying data structures, is essential for writing clear, correct, and efficient code.

### 1. Lists: Mutable, Dynamic, Homogeneous Sequences

A `list` is a mutable, ordered collection of items. Its flexibility makes it a workhorse for tasks where the size of the collection is not known beforehand or needs to change during execution.

#### 1.1 List Internals and Performance Characteristics
Understanding the performance characteristics of list methods is critical for writing efficient code. A Python list is implemented as a **dynamic array**. This means it is a contiguous block of memory that holds an array of references (pointers) to the objects it contains. This design has direct consequences:

- **Fast Indexing:** Accessing an element by index (e.g., `my_list[i]`) is very fast, an O(1) operation, because the memory location can be calculated directly from the base address and the index.
- **Fast Appending:** `my_list.append(x)` is also very fast, with an **amortized** O(1) time complexity. To avoid having to resize the array for every single append, Python's lists use a strategy of **overallocation**. When the list needs to grow, it allocates a new block of memory that is larger than immediately necessary. This new space can accommodate several future appends before another resize is needed. While a single append might occasionally trigger a costly O(n) resize-and-copy operation, the average cost over many appends is constant.
- **Slow Insertion/Deletion at the Front:** `my_list.insert(0, x)` and `my_list.pop(0)` are slow, O(n) operations. To insert an item at the beginning, all existing elements in the array must be shifted one position to the right. Similarly, popping from the front requires shifting all remaining elements to the left. 

> **Performance Guideline:** For sequences that require frequent additions or removals from *both* ends (a common pattern in queueing simulations or graph algorithms), a `collections.deque` is a much more performant choice than a `list`.

The diagram below illustrates the over-allocation strategy. When an `append` operation exceeds the list's current capacity, Python allocates a new, larger block of memory and copies the pointers from the old block to the new one, leaving spare capacity for future appends.

![Python List Over-allocation](../images/1.5-list-overallocation.png)

#### 1.2 List Comprehensions and Generator Expressions
A **list comprehension** is a concise and highly readable syntax for creating a new list by applying an expression to each item in an existing sequence. The structure `[expression for item in iterable if condition]` is generally more performant than an equivalent `for` loop because it avoids the overhead of repeated `list.append()` calls and is optimized in the CPython interpreter.

A related construct is a **generator expression**, which uses parentheses instead of square brackets: `(expression for item in iterable)`. It looks similar but is fundamentally different: instead of building a full list in memory, it creates an **iterator** that yields items one by one on demand. This is far more memory-efficient for very large sequences, as the full list is never constructed.

In [ ]:
sec("List Comprehensions and Generator Expressions")
prices = [100, 102, 101, 103, 105, 104]

note("Example 1: Calculating log returns with a list comprehension")
# zip pairs adjacent prices; the list comprehension computes log returns.
log_returns = [math.log(p2 / p1) for p1, p2 in zip(prices[:-1], prices[1:])]
print(f"Prices: {prices}")
print(f"Log Returns: {[f'{r:.4f}' for r in log_returns]}")

note("Example 2: Using a generator expression to compute the sum of squared positive returns")
# The generator expression `(r**2 for r in log_returns if r > 0)` creates an iterator.
# The sum() function consumes the iterator, never building an intermediate list in memory.
# This is highly efficient for large datasets.
sum_sq_positive = sum(r**2 for r in log_returns if r > 0)
print(f"Sum of squared positive returns: {sum_sq_positive:.4f}")

note("Example 3: Memory efficiency of generators")
N = 1_000_000
list_comp = [i for i in range(N)]
gen_exp = (i for i in range(N))
print(f"Memory for list comprehension: {sys.getsizeof(list_comp) / 1e6:.2f} MB")
print(f"Memory for generator expression: {sys.getsizeof(gen_exp)} bytes (it's a tiny object)")

#### 1.3 Slicing: Accessing Subsequences
Slicing is a powerful feature for extracting subsequences. The syntax is `my_list[start:stop:step]`.
- `start`: The index of the first item to include (default is 0).
- `stop`: The index of the first item to *exclude* (default is the end of the list).
- `step`: The amount to increment the index by (default is 1).

A slice always produces a **new list** (a shallow copy, which we will discuss shortly).

In [ ]:
sec("Slicing Lists")
years = [2018, 2019, 2020, 2021, 2022, 2023]

print(f"Original: {years}")
print(f"First three years (years[:3]): {years[:3]}")
print(f"Last two years (years[-2:]): {years[-2:]}")
print(f"Every other year (years[::2]): {years[::2]}")
print(f"Reversed list (years[::-1]): {years[::-1]}")

note("Slicing creates a shallow copy")
years_copy = years[:] # A common idiom for creating a shallow copy
print(f"'years_copy is years' is {years_copy is years} (they are different objects)")
print(f"'years_copy == years' is {years_copy == years} (their contents are equal)")

#### 1.4 Advanced Sorting
Sorting is a common operation on lists. Python provides two main ways to sort:
- `my_list.sort()`: Sorts the list **in-place** (it modifies the original list) and returns `None`.
- `sorted(my_list)`: Returns a **new, sorted list**, leaving the original list unchanged.

**The `key` Argument:** The power of Python's sorting lies in the `key` argument. This argument accepts a function that is called on each element to produce a comparison key. For simple cases, a `lambda` function is convenient. For sorting objects by attribute or items by index, `operator.itemgetter` and `operator.attrgetter` are often more readable and performant.

**Sort Stability:** Python's sort is **stable**. This means that if multiple records have the same comparison key, their original relative order is preserved in the sorted output. This is a powerful feature that allows for building up complex sorts by applying multiple sorting passes in sequence. For example, to sort a list of agents first by wealth (descending) and then by age (ascending for ties in wealth), you can perform two sorts: first, sort by the secondary key (age, ascending), and then sort that result by the primary key (wealth, descending). Because the second sort is stable, it will not disturb the relative order of agents who have the same wealth, preserving the age-based ordering established in the first pass.

In [ ]:
sec("Advanced Sorting with a Key")
# A list of named tuples for better readability
Agent = namedtuple('Agent', ['id', 'wealth', 'age'])
agents = [Agent(101, 150_000, 45), Agent(102, 80_000, 32), Agent(103, 150_000, 38)]

note("Sorting agents by wealth (descending), then age (ascending)")
# The key is a lambda function that returns a tuple.
# To achieve descending order for wealth (the primary key), we sort by its negation.
sorted_agents = sorted(agents, key=lambda agent: (-agent.wealth, agent.age))

print("Original agents:", agents)
print("Sorted agents:  ", sorted_agents)

note("Using operator.attrgetter for the same task (often faster and cleaner)")
# This is a two-pass sort, leveraging stability. First sort by age, then by wealth.
s1 = sorted(agents, key=operator.attrgetter('age')) # Sort by secondary key first
s2 = sorted(s1, key=operator.attrgetter('wealth'), reverse=True) # Then sort by primary key
print("Sorted with operator: ", s2)

#### 1.5 Augmented Assignment and Mutability
Augmented assignment operators (`+=`, `*=`) behave differently for mutable and immutable sequences. This distinction is a frequent source of bugs.
- For **immutable** sequences like tuples, `a += b` is syntactic sugar for `a = a + b`. This creates a completely **new tuple** and rebinds the name `a` to it.
- For **mutable** sequences like lists, `+=` is implemented via the `__iadd__` special method (in-place add). This **modifies the list in-place** and does not create a new object. It is more memory-efficient than the equivalent `my_list = my_list + other_list`.

This can lead to surprising behavior if a tuple contains a mutable object, like a list.

In [ ]:
sec("Augmented Assignment Behavior")

note("For a list (mutable), += modifies in-place")
list_a = [1, 2, 3]
id_before = id(list_a)
list_a += [4, 5]
id_after = id(list_a)
print(f"List content: {list_a}")
print(f"ID before: {id_before}\nID after:  {id_after}")
print(f"Object is the same: {id_before == id_after}")

note("For a tuple (immutable), += creates a new object")
tuple_a = (1, 2, 3)
id_before = id(tuple_a)
tuple_a += (4, 5)
id_after = id(tuple_a)
print(f"Tuple content: {tuple_a}")
print(f"ID before: {id_before}\nID after:  {id_after}")
print(f"Object is the same: {id_before == id_after}")

note("The dangerous edge case: a tuple containing a list")
t_anomalous = (1, [2, 3])
try:
    # This operation will fail, because tuples don't support item assignment.
    t_anomalous[1] += [4, 5]
except TypeError as e:
    print(f"Caught expected error: {e}")
    # BUT, the in-place modification of the inner list *succeeded* before the error was raised!
    print(f"Anomalous tuple is now: {t_anomalous} <-- The inner list was mutated!")

### 2. Tuples: Immutable, Heterogeneous Records

A `tuple` is an immutable, ordered collection. Its primary use case is for grouping together heterogeneous data into a single, fixed-size record where the position of each element has a distinct meaning.

Their immutability is a critical feature with two main benefits:
1.  **Hashability:** Because a tuple's value cannot change, Python can compute a unique hash value for it. This makes tuples **hashable**, meaning they can be used as keys in a dictionary or elements in a set, which is impossible for lists.
2.  **Safety:** When you pass a tuple to a function, you can be certain that the function cannot accidentally modify its contents.

#### 2.1 Sequence Unpacking
**Unpacking** is a powerful feature that allows the assignment of items in a sequence to multiple variables in a single, readable statement. It is particularly effective with tuples, where each position corresponds to a specific field. The `*` operator can be used to grab excess items into a list, providing flexibility when dealing with sequences of unknown length.

In [ ]:
sec("Tuple Unpacking")
country_gdp_record = ('USA', 2022, 25.46, 'trillion')

# The * operator allows for flexible unpacking, collecting extra items into a list.
country, year, *values = country_gdp_record

print(f"Record: {country_gdp_record}")
print(f"Country: {country}")
print(f"Year: {year}")
print(f"Values: {values}")

#### 2.2 Named Tuples for Self-Documenting Code
A limitation of standard tuples is that data can only be accessed by a numeric index (e.g., `record[1]`), which can make code hard to read and prone to off-by-one errors. The `collections.namedtuple` factory function addresses this by creating custom tuple subclasses that allow fields to be accessed by name, like an object attribute. This provides the best of both worlds: the memory efficiency and immutability of a tuple, combined with the readability of object-like access.

In modern Python, `typing.NamedTuple` is often preferred as it provides the same functionality but with support for type hints, which improves code clarity and allows for static analysis.

In [ ]:
sec("typing.NamedTuple for Readable and Type-Hinted Records")
# Define a 'ModelParams' class for a standard RBC model using the modern syntax
class ModelParams(NamedTuple):
    alpha: float  # Capital share
    beta: float   # Discount factor
    delta: float  # Depreciation rate

rbc_params = ModelParams(alpha=0.33, beta=0.99, delta=0.025)

print(f"Model parameters object: {rbc_params}")
note("Accessing data by name is explicit and less error-prone.")
print(f"The discount factor (beta) is: {rbc_params.beta}")

note("Named tuples are still regular tuples and support indexing and unpacking.")
print(f"Alpha by index: {rbc_params[0]}")
alpha, beta, delta = rbc_params # Unpacking works as expected
print(f"Unpacked alpha: {alpha}")

### 3. The Python Memory Model: References, Copies, and Mutability

A frequent source of bugs in computational work is a misunderstanding of Python's memory model. In Python, variables are not containers for values; they are **names** or **labels** that point to objects residing in memory. The assignment operator (`=`) **never copies an object**. It simply attaches a new name to the same object.

A Python `list` object in memory does not store the actual data items contiguously. Instead, it holds an array of **references** (memory addresses), which in turn point to the actual objects (like the integers 10, 20, 30) stored elsewhere in memory. This design is flexible, as it allows a list to hold objects of different types, but it has important implications for memory usage and copying that we explore next.

#### 3.1 Assignment Creates an Alias
When you write `list_b = list_a`, you are not creating a new list. You are creating a second name, `list_b`, that points to the *exact same list object* as `list_a`. Any modification made through one name will be visible through the other, because they are two names for one object. We can verify this using the built-in `id()` function, which returns the unique memory address of an object.

In [ ]:
sec("Assignment Creates an Alias")
scenario_A = [0.025, 0.99] # [depreciation, discount_factor]
scenario_B = scenario_A   # B is an alias for A, not a copy

note(f"Initially: Scenario A is {scenario_A}, Scenario B is {scenario_B}")
print(f"  Memory ID of A: {id(scenario_A)}")
print(f"  Memory ID of B: {id(scenario_B)}")
note(f"The IDs are identical. 'scenario_A is scenario_B' is {scenario_A is scenario_B}.")

scenario_B[0] = 0.05 # Change depreciation in Scenario B
note("After modifying Scenario B...")
print(f"  Scenario B is now: {scenario_B}")
print(f"  Scenario A is also: {scenario_A} <-- The single underlying object was changed.")

#### 3.2 Shallow Copies for Nested Structures
A **shallow copy** (created via `list.copy()` or slicing `[:]`) creates a new top-level list object. This new list is a distinct object in memory. However, the new list is populated with references to the *same objects* that were contained in the original list. 

A **shallow copy** creates a new top-level list, but this new list is populated with references pointing to the *same objects* contained in the original. This is perfectly safe if the list contains only immutable objects (like numbers or strings). However, if the list contains other mutable objects (like other lists), this can lead to subtle bugs, as modifying a nested object via the copy will also modify it in the original.

In [ ]:
sec("Shallow Copy with Nested Lists")
# List of policy vectors: [[tax_rate, subsidy_level], ...]
baseline_policy = [[0.20, 0.05], [0.15, 0.10]]
reform_policy = baseline_policy.copy() # Shallow copy

note(f"The top-level lists are different objects: 'baseline_policy is reform_policy' is {baseline_policy is reform_policy}.")
note(f"But the nested lists are the same: 'baseline_policy[0] is reform_policy[0]' is {baseline_policy[0] is reform_policy[0]}.")

note("Modifying a nested element (the tax rate of the first vector) in the reform policy...")
reform_policy[0][0] = 0.25

print(f"  Reform Policy:   {reform_policy}")
print(f"  Baseline Policy: {baseline_policy} <-- CORRUPTED! The inner list object was shared.")

#### 3.3 Deep Copies for Full Independence
A **deep copy**, created using the `copy.deepcopy()` function, resolves this issue by creating a fully independent replica. It recursively traverses the original object and creates new copies of all objects it finds, no matter how deeply they are nested. This is the safest way to ensure that two complex, nested objects are completely decoupled.

In [ ]:
sec("Deep Copy for True Independence")
baseline_policy = [[0.20, 0.05], [0.15, 0.10]]
reform_policy = copy.deepcopy(baseline_policy)

note(f"Now, the nested lists are also different objects: 'baseline_policy[0] is reform_policy[0]' is {baseline_policy[0] is reform_policy[0]}.")

note("Modifying a nested element in the deep copy...")
reform_policy[0][0] = 0.25

print(f"  Reform Policy:   {reform_policy}")
print(f"  Baseline Policy: {baseline_policy} <-- SAFE. The copy is fully independent.")

> **Rule of Thumb:** When creating a copy of a list, use a shallow copy (`.copy()` or `[:]`) if the list contains only immutable objects (like numbers, strings, or tuples of immutables). If the list contains *any* mutable objects (like other lists or custom class instances), use `copy.deepcopy()` to prevent unintended side effects.

### 4. Advanced and Specialized Sequences

While `list` and `tuple` are the workhorses, Python's standard library offers specialized sequence types optimized for specific use cases.

#### 4.1 Deques: High-Performance Double-Ended Queues

The `collections.deque` (pronounced "deck") is a specialized mutable sequence optimized for fast appends and pops from **both ends**.

##### Conceptual Implementation and Performance Trade-offs
To understand when to use a `deque`, it's useful to have a mental model of its internal implementation compared to a `list`.

- **`list`**: Implemented as a **dynamic array**, which is a single, contiguous block of memory holding pointers to the items. 
  - **Analogy:** A row of seats in a theater. Accessing any seat by its number (`my_list[i]`) is instantaneous (O(1)). But if the person at the front leaves (`.pop(0)`), everyone else must shift down one seat, which is a slow, O(n) process for a large list.

- **`deque`**: Implemented as a **doubly-linked list of fixed-size blocks of items**.
  - **Analogy:** A train made of linked carriages. Adding or removing a carriage from the front (`.appendleft()`, `.popleft()`) or back (`.append()`, `.pop()`) is very fast (O(1)) because you only need to change the links on the engine or caboose. However, finding the 100th passenger (`my_deque[100]`) requires walking through the carriages from one end, which is an O(n) operation, making it slower than direct access in a list for elements in the middle.

| Operation              | `list` Complexity | `deque` Complexity | Winner for this Task        |
|------------------------|-------------------|--------------------|-----------------------------|
| Append to right        | O(1)              | O(1)               | (Tie)                       |
| Pop from right         | O(1)              | O(1)               | (Tie)                       |
| Append to left         | **O(n)**          | **O(1)**           | **`deque`** (by a lot)      |
| Pop from left          | **O(n)**          | **O(1)**           | **`deque`** (by a lot)      |
| Access element by index| O(1)              | O(n)               | `list`                      |

**Guideline:** Use a `deque` whenever you have a queuing or buffering problem that requires frequent additions or removals from both ends (FIFO or LIFO). Use a `list` when you need fast random access to elements by index.

##### Performance Comparison: The Cost of Shifting

The theoretical difference between an O(n) and an O(1) operation has dramatic real-world consequences, especially for large sequences. The following code uses the `timeit` module to precisely measure the time taken to repeatedly remove items from the front of a `list` versus a `deque`.

In [ ]:
sec("Performance: list.pop(0) vs. deque.popleft()")

n = 150_000
list_setup = f"import collections; data = list(range({n}))"
deque_setup = f"import collections; data = collections.deque(range({n}))"

list_code = "while data: data.pop(0)"
deque_code = "while data: data.popleft()"

# timeit runs the code multiple times to get a reliable measurement
list_time = timeit.timeit(list_code, setup=list_setup, number=3)
deque_time = timeit.timeit(deque_code, setup=deque_setup, number=3)

print(f"Time to drain a list of size {n:,} from the left:  {list_time:.4f} seconds")
print(f"Time to drain a deque of size {n:,} from the left: {deque_time:.4f} seconds")

if deque_time > 0:
    note(f"The deque was approximately {list_time / deque_time:,.0f}x faster for this O(n) vs O(1) operation.")

##### Code Lab: Efficiently Calculating a Moving Average

A common task in time series analysis is calculating a moving average to smooth out data. A `deque` with a fixed maximum length (`maxlen`) is the ideal data structure for this task. When a new item is appended to a full `deque`, an item is automatically and efficiently discarded from the opposite end. This creates a perfect "sliding window" of recent observations, which is far more efficient than manually slicing a list at each time step.

The code below defines a generator function that takes a stream of data and yields the moving average as the window slides along.

In [ ]:
sec("Calculating a Moving Average with a Deque")

def moving_average_stream(data_stream, *, window_size):
    """A generator that yields the moving average of a data stream.
    Uses a deque with maxlen for an efficient sliding window.
    """
    # Initialize the deque with a fixed maximum size. This is the key to the pattern.
    window = deque(maxlen=window_size)
    
    for new_value in data_stream:
        # Appending to the right automatically and efficiently pushes the oldest element 
        # off the left once the deque is full. This is an O(1) operation.
        window.append(new_value)
        
        # Yield the average only when the window has filled up.
        if len(window) == window_size:
            yield sum(window) / window_size

# Simulate a stream of noisy price data
np.random.seed(42)
price_stream = 100 + np.random.randn(50).cumsum()

# The result is a list of the moving average values
moving_avg = list(moving_average_stream(price_stream, window_size=10))

note("Plotting the raw price data against its 10-period moving average.")

plt.figure(figsize=(10, 5))
plt.plot(price_stream, label='Raw Price', color='gray', alpha=0.7, linestyle='--')
# The moving average series will be shorter than the original by (window_size - 1),
# so we must align the x-axis for plotting.
plt.plot(range(9, 50), moving_avg, label='10-Period Moving Average', color='firebrick', linewidth=2)
plt.title('Smoothing Time Series Data with a Deque-Based Moving Average')
plt.xlabel('Time Period'); plt.ylabel('Price'); plt.legend(); plt.grid(True)
plt.tight_layout()
plt.show()

#### 4.2 Arrays: Memory-Efficient Numeric Sequences
The built-in `array.array` object provides a more memory-efficient way to store a sequence of homogeneous numeric data. Unlike a `list`, which stores references to full-fledged Python `int` or `float` objects, an `array` stores the raw bytes of the numbers directly, similar to an array in C. This can result in significant memory savings, especially for large sequences of numbers.

You must specify a type code upon creation to determine the type of data it will hold (e.g., `'i'` for signed integer, `'d'` for double-precision float).

> **Note:** For most scientific and economic computing, a `numpy.ndarray` is far more powerful and versatile than an `array.array`. However, understanding `array.array` is useful for historical context and for situations where you need a memory-efficient sequence without pulling in the entire NumPy dependency.

In [ ]:
sec("Memory Efficiency of array.array")
N = 1_000_000
list_of_floats = [float(i) for i in range(N)]
array_of_floats = array.array('d', list_of_floats)

note(f"Memory usage of a list of {N:,} floats: {sys.getsizeof(list_of_floats) / 1e6:.2f} MB")
note(f"Memory usage of an array of {N:,} floats: {sys.getsizeof(array_of_floats) / 1e6:.2f} MB")

#### 4.3 `bisect`: Maintaining Sorted Sequences

The `bisect` module is a powerful tool for working with sorted sequences. Its functions use the binary search algorithm to find insertion points for new items, which is an O(log n) operation. This is vastly more efficient than repeatedly sorting a list (O(n log n)) or finding the insertion point manually (O(n)).

This module is highly relevant for economic modeling, particularly in tasks like:
- Building a cumulative distribution function (CDF) from a set of observations.
- Finding the correct interval on a discretized grid for interpolation.
- Efficiently inserting new events into a sorted timeline in a simulation.

**Core Functions:**
- `bisect.bisect_left(a, x)`: Finds the insertion point for `x` in the sorted sequence `a` to maintain sort order. If `x` is already present, the insertion point will be to the left of existing entries.
- `bisect.insort_left(a, x)`: Inserts `x` into `a` at the correct position. This is the combination of `bisect_left` and `list.insert`.

In [ ]:
import bisect
sec("`bisect` for Efficient Insertion and Searching")

# Imagine we have a sorted grid representing income brackets
income_brackets = [0, 15000, 45000, 90000, 150000, 500000]

note("Finding the correct bracket for a new income value")
new_income = 75000
# bisect_left returns the index where the item should be inserted.
# This index - 1 gives us the bracket it falls into.
idx = bisect.bisect_left(income_brackets, new_income)
print(f"An income of ${new_income:,} falls into bracket {idx-1}, which starts at ${income_brackets[idx-1]:,}.")

note("Inserting new observations into a sorted list of returns")
returns = [-0.02, 0.01, 0.015, 0.03]
new_return = 0.005
print(f"Original returns: {returns}")
bisect.insort_left(returns, new_return)
print(f"After inserting {new_return}: {returns}")

### 5. Exercises

#### Exercise 1: Sorting Complex Records
a. Define a `typing.NamedTuple` called `CountryData` with fields `name` (str), `gdp_pcap` (float), and `gini` (float).
b. Create a list of at least four `CountryData` instances with some sample data.
c. Use a multi-pass, stable sort with `operator.attrgetter` to sort this list. The primary sorting key should be the Gini coefficient in ascending order (lowest inequality first). The secondary sorting key should be GDP per capita in descending order. Print the sorted list.

#### Exercise 2: The Copying Puzzle
Consider the following setup:
```python
import copy
policy_A = [0.2, [50, 10]] # [tax_rate, [benefit_low_income, benefit_high_income]]
policy_B = policy_A
policy_C = policy_A.copy()
policy_D = copy.deepcopy(policy_A)

# Now, we apply a change
policy_C[0] = 0.25
policy_C[1][0] = 60
```
Without running the code, predict the final values of `policy_A`, `policy_B`, `policy_C`, and `policy_D`. Write down your predictions and then verify them by running the code. Explain *why* `policy_A` was affected by the change to `policy_C[1][0]` but not by the change to `policy_C[0]`.

#### Exercise 3: Performance: `list` vs. `deque`
- **Task:** Write a short script to measure the time it takes to perform 100,000 insertions at the beginning of a sequence using both a `list` (`my_list.insert(0, i)`) and a `collections.deque` (`my_deque.appendleft(i)`). 
- **Analysis:** Use the `time.perf_counter()` function to time each loop. Report the time taken for each data structure and explain the performance difference based on their underlying implementations (dynamic array vs. doubly-linked list).

--- 

### Challenge Exercise: Interpolating from a Sorted Sequence

A common task in economics is to look up a value from a table or grid, such as finding a tax rate for a given income. This exercise combines `NamedTuple` for structured data, sorting, and the `bisect` module for efficient searching.

**Objective:** Create a simple tax bracket lookup system.

1.  **Define the Data Structure:**
    a. Create a `typing.NamedTuple` called `TaxBracket` with fields `income_threshold` (float) and `marginal_rate` (float).

2.  **Create and Sort the Data:**
    a. Create a list of `TaxBracket` instances representing a progressive tax system (e.g., `[TaxBracket(0, 0.10), TaxBracket(20000, 0.15), TaxBracket(80000, 0.25), ...]`). The list does not need to be initially sorted.
    b. Sort this list in-place based on the `income_threshold`.

3.  **Build the Lookup Function:**
    a. Write a function `get_tax_rate(income: float, brackets: List[TaxBracket]) -> float`.
    b. Inside the function, you cannot iterate through the whole list (that would be O(n)). You must use the `bisect` module to perform an efficient O(log n) lookup.
    c. `bisect.bisect_left` will not work directly on your list of objects. You will need to create a list of just the `income_threshold` values to search over.
    d. Use the index returned by `bisect` to find the correct bracket. Remember that `bisect` finds the insertion point, so the correct bracket will be at `index - 1`.
    e. Handle the edge case where the income is below the first threshold.

4.  **Demonstrate:**
    a. Test your function with several different income levels to show that it returns the correct marginal tax rate.